In [35]:
import numpy as np
from data_named import data
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

print(data.columns.values)
print(data.shape)

['Duration (in seconds)' 'UserLanguage' 'participant_number' 'age'
 'gender' 'ethnicity' 'political_orientation' 'political_orientation_text'
 'achieved_education' 'achieved_education_text' 'bothered_pre'
 'uneasy_pre' 'uncomfortable_pre' 'C_time_alone' 'C_academic_performance'
 'T_time_alone' 'T_academic_performance' 'value_activation'
 'bothered_post' 'uneasy_post' 'uncomfortable_post' 'sandwich' 'diet'
 'diet_text' 'reason_for_choice' 'labels_attention' 'feeling_observed'
 'did_value_activation_affect_choice']
(18, 28)


![alternative text](med_analysis.png)

From Rijnhart & Al. 2023 using their equations:

![alternative text](med_analysis_eq.png)

Mediation Analysis Part 1 (eq. 1)

In [19]:
#data = data.drop(['Duration (in seconds)', 'UserLanguage', 'political_orientation_text', ], axis=1)

sandwiches = data['sandwich'].to_numpy()
Y = np.where(sandwiches < 3, 1, -1)

participant_nb = data['participant_number'].to_numpy()
x = np.where(participant_nb > 70, 1, -1)

logreg = LogisticRegression()
logreg.fit(x.reshape(-1, 1), Y)

a = logreg.intercept_[0]
b = logreg.coef_[0][0]

print("Intercept : " + str(a) + ", Coef. : " + str(b))

Intercept : -0.3130665241541008, Coef. : 0.5957788357720568


Mediation Analysis Part 2 (eq. 2)

In [27]:
M1 = data['bothered_post'].to_numpy() - data['bothered_pre'].to_numpy()
M2 = data['uneasy_post'].to_numpy() - data['uneasy_pre'].to_numpy()
M3 = data['uncomfortable_post'].to_numpy() - data['uncomfortable_pre'].to_numpy()

logreg.fit(x.reshape(-1, 1), M1)
a1 = logreg.intercept_[0]
b1 = logreg.coef_[0][0]

logreg.fit(x.reshape(-1, 1), M2)
a2 = logreg.intercept_[0]
b2 = logreg.coef_[0][0]

logreg.fit(x.reshape(-1, 1), M3)
a3 = logreg.intercept_[0]
b3 = logreg.coef_[0][0]

print("Bothered (M1)")
print("Intercept : " + str(a1) + ", Coef. : " + str(b1))
print()
print("Uneasy (M2)")
print("Intercept : " + str(a2) + ", Coef. : " + str(b2))
print()
print("Uncomfortable (M3)")
print("Intercept : " + str(a3) + ", Coef. : " + str(b3))

Bothered (M1)
Intercept : -0.3364950388496827, Coef. : 0.2622532842846421

Uneasy (M2)
Intercept : -0.23982866589368165, Coef. : 0.46631417967444866

Uncomfortable (M3)
Intercept : -0.36619944920871833, Coef. : 0.4341941225620249


Mediation Analysis Part 3 (eq. 3)

In [29]:
X = np.column_stack((x, M1, M2, M3))

logreg.fit(X, Y)

intercept = logreg.intercept_
coefficients = logreg.coef_

print("Intercept:", intercept)
print("Coefficients:")
print("VA (x):", coefficients[0][0])
print("Bothered (M1):", coefficients[0][1])
print("Uneasy (M2):", coefficients[0][2])
print("Uncomfortable (M3):", coefficients[0][3])


Intercept: [-0.43147906]
Coefficients:
VA (x): 0.46434185079174545
Bothered (M1): 0.08455677845224968
Uneasy (M2): 0.09401341487460245
Uncomfortable (M3): 0.018430097659608763
